In [1]:
import pandas as pd

In [3]:
df_taxi = pd.read_csv('data/Order_Taxi.csv', header=[1])
df_taxi51 = pd.read_csv('data/Order_Taxi_51.csv', header=[1])
df_tour = pd.read_csv('data/Order_Touring.csv', header=[1])
df_tour51 = pd.read_csv('data/Order_Touring_51.csv', header=[1])

In [9]:
df_taxi.head(3)

,ORDER_ID,ON_AREA,VEHICLE_NO,BOOK_DEP_TIME,WAIT_TIME,DEP_LONGITUDE,DEP_LATITUDE,DEP_AREA,DEP_TIME,DEST_LONGITUDE,DEST_LATITUDE,DEST_AREA,DEST_TIME,DRIVE_MILE,DRIVE_TIME,WAIT_MILE,ORDER_MATCH_TIME
0,6511269774274249728,350200,25A2,20180115213211,0.0,118.148247,24.543191,厦门高崎国际机场,20180115214118,118.089052,24.442635,和驿家花园酒店-西南门,20180115220957,24.9,1719,0.0,20180115220957
1,35197522115351,350211,6632,20190704224545,NaN,118.079849,24.637237,万科广场幸福里-北门,20190704224720,118.101534,24.579060,石鼓路-|集美大学轮机工程学院,20190704225919,8.9,660,NaN,20190706235810
2,35197574629706,350203,5856,20190706234640,NaN,118.112412,24.471414,湖滨南路-|厦门万象城-5号门-上车点,20190706234927,118.075673,24.463243,鹭江|华侨海景城1期,20190706235809,3.9,480,NaN,20190706235811


In [14]:
import geopandas as gpd
from geopandas.tools import sjoin

In [15]:
cname_long = 'DEP_LONGITUDE'
cname_lat = 'DEP_LATITUDE'

In [16]:
gdf_taxi = gpd.GeoDataFrame(
    df_taxi, 
    geometry=gpd.points_from_xy(
        df_taxi[cname_long], 
        df_taxi[cname_lat])
)
gdf_taxi.head()

,ORDER_ID,ON_AREA,VEHICLE_NO,BOOK_DEP_TIME,WAIT_TIME,DEP_LONGITUDE,DEP_LATITUDE,DEP_AREA,DEP_TIME,DEST_LONGITUDE,DEST_LATITUDE,DEST_AREA,DEST_TIME,DRIVE_MILE,DRIVE_TIME,WAIT_MILE,ORDER_MATCH_TIME,geometry
0,6511269774274249728,350200,25A2,20180115213211,0.0,118.148247,24.543191,厦门高崎国际机场,20180115214118,118.089052,24.442635,和驿家花园酒店-西南门,20180115220957,24.9,1719,0.0,20180115220957,POINT (118.14825 24.54319)
1,35197522115351,350211,6632,20190704224545,NaN,118.079849,24.637237,万科广场幸福里-北门,20190704224720,118.101534,24.579060,石鼓路-|集美大学轮机工程学院,20190704225919,8.9,660,NaN,20190706235810,POINT (118.07985 24.63724)
2,35197574629706,350203,5856,20190706234640,NaN,118.112412,24.471414,湖滨南路-|厦门万象城-5号门-上车点,20190706234927,118.075673,24.463243,鹭江|华侨海景城1期,20190706235809,3.9,480,NaN,20190706235811,POINT (118.11241 24.47141)
3,35197573821494,350212,3533,20190706224956,NaN,118.149850,24.635240,滨安路-|厦门市-美林中学,20190706225021,118.148172,24.625230,同安区-|彩虹沙滩,20190706225349,3.0,480,NaN,20190706235812,POINT (118.14985 24.63524)
4,17635117866582,350203,17756,20190704234701,NaN,118.120209,24.474520,莲坂|国骏大厦西北侧,20190704234840,118.141514,24.495640,仙岳路-沿线|新景华府,20190704235757,5.0,540,NaN,20190706235812,POINT (118.12021 24.47452)


In [18]:
gdf_xm_district = gpd.read_file('data/district/xm.geojson')

pointInPolys = sjoin(gdf_taxi, gdf_xm_district, how='left')
grouped = pointInPolys.groupby('index_right')

/Users/arthur/opt/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [19]:
for key, item in grouped:
    print(key, len(item))

0.0 62
1.0 273
2.0 186
3.0 392
4.0 108
5.0 70


In [22]:
import folium
from folium.plugins import FastMarkerCluster

lat = 24.651386
long = 118.129117

# define xm map
map_xm = folium.Map(location=[lat, long], zoom_start=10)
folium.GeoJson('data/district/xm.geojson', name='geojson').add_to(map_xm)

# list_list_pois = gdf_taxi[[cname_lat, cname_long]].values.tolist()

# Faster Marker cluster
for key, item in grouped:
    FastMarkerCluster(data=item[[cname_lat, cname_long]].values.tolist()).add_to(map_xm)


map_xm